In [1]:
import numpy.random as nr
from math import exp, pi, sqrt
from prettytable import PrettyTable

In [2]:
class UniformGenerator:
    def __init__(self, a, b):
        if not 0 <= a <= b:
            raise ValueError('Параметры должны удовлетворять условию 0 <= a <= b')
        self._a = a
        self._b = b

    def next(self):
        return nr.uniform(self._a, self._b)

In [3]:
class PoissonGenerator:
    def __init__(self, la):
        self._la = la
    
    def next(self):
        return nr.poisson(self._la)

In [4]:
class ConstGenerator:
    def __init__(self, c):
        self._c = c
        
    def next(self):
        return self._c

In [13]:
class RandomFnGenerator:
    def __init__(self, fn):
        self._fn = fn
        
    def next(self, t = 0, dt = 0.1):
        rt = t + dt
        while nr.random_sample() > self._fn(rt / (60)) and rt < 480:
            rt += dt
#         print(self._fn(rt / (60 / dt)))
        return rt - t

In [6]:
class RandomIntGenerator:
    def __init__(self, a, b):
        self._a = a
        self._b = b
        
    def next(self):
        return nr.randint(a, b + 1)

In [7]:
class Generator:
    def __init__(self, generator):
        self._generator = generator
        self._receivers = []
        self._generated_requests = 0
        self._next_event_time = 0
        
    @property
    def next_event_time(self):
        return self._next_event_time
    
    @next_event_time.setter
    def next_event_time(self, time):
        self._next_event_time = time
        
    @property
    def generated_requests(self):
        return self._generated_requests
    
    @property
    def receivers(self):
        return self._receivers
        

    def add_receiver(self, receiver):
        if receiver not in self._receivers:
            self._receivers.append(receiver)

    def remove_receiver(self, receiver):
        try:
            self._receivers.remove(receiver)
        except KeyError:
            pass
    
    def next_time(self, p1 = 0, p2 = 0.1):
        try:
            return self._generator.next(p1, p2)
        except:
            return self._generator.next()
    
    def emit_request(self):
        if (len(self._receivers) == 0):
            return None
        
        self._generated_requests += 1
        if 'cook' in self._receivers[0].name:
            self._receivers.sort(key = (lambda x: x.next_event_time))
            
        for rec in self._receivers:
            if rec.receive_request():
                return rec
        return None

In [8]:
class Queue:
    def __init__(self, max_size):
        self._queued = 0
        self._max_queue_size = max_size
        self._queue_size = max_size
        self._queue = []
        
    @property
    def max_queue_size(self):
        return self._max_queue_size

    @property
    def current_queue_size(self):
        return self._queue_size
    
    @property
    def queued(self):
        return self._queued
    
    @property
    def current(self):
        if (len(self._queue) == 0):
            return -1
        return self._queue[0]
    
    def decrease(self):
        if (len(self._queue) == 0):
            return -1
        self._queue[0] -= 1
    
    def add(self, el = 1):
        self._queued += 1
        self._queue.append(el)
        
    def remove(self):
        self._queued -= 1
        self._queue.pop(0)
        
    def increase_size(self):
        self._queue_size += 1

In [9]:
class Processor(Generator):
    def __init__(self, generator, queue, *, return_probability = 0.0, name = ''):
        super().__init__(generator)
        self._generator = generator
        self._processed_requests = 0
        self._return_probability = return_probability
        self._reentered_requests = 0
        self._queue = queue
        self._name = name
        
    @property
    def processed_requests(self):
        return self._processed_requests
    
    @property
    def reentered_requests(self):
        return self._reentered_requests
    
    @property
    def queue(self):
        return self._queue
    
    @property
    def name(self):
        return self._name
    
    def process(self):
        if self._queue.queued > 0:
            self._processed_requests += 1
#             self._queue.remove()
            if self._remove_from_queue():
                assigned_processor = self.emit_request()
                if nr.random_sample() < self._return_probability:
                    self._reentered_requests += 1
                    self.receive_request()
                return assigned_processor
            return None

    def receive_request(self):
        if self._queue.max_queue_size == 0:
            if self._queue.queued >= self._queue.current_queue_size:
                self._queue.increase_size()
#             self._queue.add()
            self._add_to_queue()
            return True
        elif self._queue.queued < self._queue.current_queue_size:
#             self._queue.add()
            self._add_to_queue()
            return True
        return False
    
    def _add_to_queue(self):
        if 'cook' in self._name:
            self._queue.add(nr.randint(1, 4 + 1))
        else:
            self._queue.add()
            
    def _remove_from_queue(self):
        if 'cook' in self._name:
            if self._queue.current > 1:
                self._queue.decrease()
                return False
            else:
                self._queue.remove()
                return True
        else:
            self._queue.remove()
            return True

    def next_time_period(self):
        return self._generator.next()

In [14]:
def modelling(max_requests = 300):
    cdesk_m = 6
    cdesk_d = 2
    edesk_m = 5
    edesk_d = 1
    cook1_m = 2
    cook1_d = 0.5
    cook2_m = 1.5
    cook2_d = 0.5
    cook3_m = 3
    cook3_d = 1
    cook4_m = 2
    cook4_d = 1
    disp_c = 0.5
    
    desk_queue_size = 3
    
    clients = Generator(RandomFnGenerator(lambda t: 0.4 / sqrt(2 * pi) * exp(-((t - 5) ** 2) / 2)))
    cdesk1 = Processor(UniformGenerator(cdesk_m - cdesk_d, cdesk_m + cdesk_d), Queue(desk_queue_size), name = 'cdesk1')
    cdesk2 = Processor(UniformGenerator(cdesk_m - cdesk_d, cdesk_m + cdesk_d), Queue(desk_queue_size), name = 'cdesk2')
    edesk1 = Processor(UniformGenerator(edesk_m - edesk_d, edesk_m + edesk_d), Queue(desk_queue_size), name = 'edesk1')
    edesk2 = Processor(UniformGenerator(edesk_m - edesk_d, edesk_m + edesk_d), Queue(desk_queue_size), name = 'edesk2')
    edesk3 = Processor(UniformGenerator(edesk_m - edesk_d, edesk_m + edesk_d), Queue(desk_queue_size), name = 'edesk3')
    edesk4 = Processor(UniformGenerator(edesk_m - edesk_d, edesk_m + edesk_d), Queue(desk_queue_size), name = 'edesk4')
    cook_queue = Queue(0)
    cook1 = Processor(UniformGenerator(cook1_m - cook1_d, cook1_m + cook1_d), cook_queue, name = 'cook1')
    cook2 = Processor(UniformGenerator(cook2_m - cook2_d, cook2_m + cook2_d), cook_queue, name = 'cook2')
    cook3 = Processor(UniformGenerator(cook3_m - cook3_d, cook3_m + cook3_d), cook_queue, name = 'cook3')
    cook4 = Processor(UniformGenerator(cook4_m - cook4_d, cook4_m + cook4_d), cook_queue, name = 'cook4')
    disp = Processor(ConstGenerator(disp_c), Queue(0))
    
    clients.add_receiver(cdesk1)
    clients.add_receiver(cdesk2)
    clients.add_receiver(edesk1)
    clients.add_receiver(edesk2)
    clients.add_receiver(edesk3)
    clients.add_receiver(edesk4)
    
    cdesk1.add_receiver(cook1)
    cdesk1.add_receiver(cook2)
    cdesk1.add_receiver(cook3)
    cdesk1.add_receiver(cook4)
    
    cdesk2.add_receiver(cook1)
    cdesk2.add_receiver(cook2)
    cdesk2.add_receiver(cook3)
    cdesk2.add_receiver(cook4)
    
    edesk1.add_receiver(cook1)
    edesk1.add_receiver(cook2)
    edesk1.add_receiver(cook3)
    edesk1.add_receiver(cook4)
    
    edesk2.add_receiver(cook1)
    edesk2.add_receiver(cook2)
    edesk2.add_receiver(cook3)
    edesk2.add_receiver(cook4)
    
    edesk3.add_receiver(cook1)
    edesk3.add_receiver(cook2)
    edesk3.add_receiver(cook3)
    edesk3.add_receiver(cook4)
    
    edesk4.add_receiver(cook1)
    edesk4.add_receiver(cook2)
    edesk4.add_receiver(cook3)
    edesk4.add_receiver(cook4)
    
    cook1.add_receiver(disp)
    cook2.add_receiver(disp)
    cook3.add_receiver(disp)
    cook4.add_receiver(disp)
    
    nodes = [clients, cdesk1, cdesk2, edesk1, edesk2, edesk3, edesk4, cook1, cook2, cook3, cook4, disp]
    for n in nodes:
        n.next_event_time = 0
    
    time = 0
    dt = 0.1
    
    aborted = 0
    left = 0
    
    clients.next_event_time = clients.next_time(0.1)
    next_event = clients.next_event_time
    
    while time < 8 * 60:
        if (next_event <= time):
            for n in nodes:
                if time >= n.next_event_time and n.next_event_time != 0:
                    if not isinstance(n, Processor):
                        if all(r.queue.queued > 0 for r in clients.receivers) and nr.random_sample() < 0.3:
                            left += 1
                        else:
                            assigned_processor = clients.emit_request()
                            if assigned_processor is not None:
                                assigned_processor.next_event_time = (n.next_event_time +
                                                                      assigned_processor.next_time())
                            else:
                                aborted += 1
                        clients.next_event_time = n.next_event_time + clients.next_time(time)
                    else:
                        assigned_processor = n.process()
                        if assigned_processor is not None:
                            assigned_processor.next_event_time = (n.next_event_time +
                                                                  assigned_processor.next_time())
                        if n.queue.queued == 0:
                            n.next_event_time = 0
                        else:
                            n.next_event_time = time + n.next_time()
            next_event = 0
#             print('next_event')
            for n in nodes:
#                 print(next_event, n.next_event_time)
                if next_event >= n.next_event_time and n.next_event_time != 0:
                    next_event = n.next_event_time
        
        time += dt
        
    print(clients.generated_requests,
          cdesk1.processed_requests,
          cook1.processed_requests + cook2.processed_requests + cook3.processed_requests + cook4.processed_requests,
          cook1.queue.queued,
          disp.processed_requests)
    print(aborted, left)
    return [disp.processed_requests,
            cook1.processed_requests + cook2.processed_requests + cook3.processed_requests + cook4.processed_requests,
            aborted + left]

In [11]:
def print_results(proc, pos, left):
    table = PrettyTable()
    table.field_names = ["", "Обслужено клиентов", "Продано позиций", "Клиентов ушло"]
    table.add_row(["min", proc[0], pos[0], left[0]])
    table.add_row(["max", proc[1], pos[1], left[1]])
    print(table)

In [15]:
min_proc = float('inf')
max_proc = 0
min_pos = float('inf')
max_pos = 0
min_left = float('inf')
max_left = 0

for i in range(10):
    res = modelling()
    if res[0] < min_proc:
        min_proc = res[0]
    if res[0] > max_proc:
        max_proc = res[0]
    if res[1] < min_pos:
        min_pos = res[1]
    if res[1] > max_pos:
        max_pos = res[1]
    if res[2] < min_left:
        min_left = res[2]
    if res[2] > max_left:
        max_left = res[2]
        
print_results([min_proc, max_proc], [min_pos, max_pos], [min_left, max_left])

191 40 411 15 157
17 52
173 42 404 4 159
10 38
190 41 422 0 177
13 38
192 40 416 7 172
12 39
207 39 399 17 165
25 52
212 42 395 36 152
23 47
200 40 407 14 163
23 60
210 43 441 0 181
29 45
192 43 397 31 151
10 50
210 42 387 31 148
30 44
+-----+--------------------+-----------------+---------------+
|     | Обслужено клиентов | Продано позиций | Клиентов ушло |
+-----+--------------------+-----------------+---------------+
| min |        148         |       387       |       48      |
| max |        181         |       441       |       83      |
+-----+--------------------+-----------------+---------------+


In [12]:
t = 0
for i in range(480):
    if nr.random_sample() < 0.4 / sqrt(2 * pi) * exp(-((i/60 - 5) ** 2) / 2):
        print(0.4 / sqrt(2 * pi) * exp(-((i/60 - 5) ** 2) / 2))
        t += 1
print(t)

0.010903172388225258
0.06271728735420344
0.08714087081302023
0.09678828980765736
0.10802764050856473
0.1249015733467045
0.1332898411567199
0.1371775420077536
0.1431133716671876
0.1584945896996621
0.1592226905255809
0.15948828294199083
0.15948828294199083
0.1592226905255809
0.15878101899080474
0.1584945896996621
0.15641707759018236
0.15177317324677816
0.1509532910771973
0.14920240272293145
0.12777920220894087
0.11276475016412107
0.09517530038031119
0.09195285627369325
0.06854743681912297
0.04436833387178226
0.02708699684944714
0.003316976886283074
28
